In [ ]:
!python3 -m pip install opencv-python matplotlib numpy


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import glob


In [ ]:
IMG_SIZE = 28

def preprocess_image(img_path):
    """
    Convierte una imagen a escala de grises, la binariza,
    la redimensiona a 28x28 y la normaliza al rango [0,1].
    """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"No se pudo leer la imagen: {img_path}")
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    _, img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    img = img / 255.0
    return img

def load_dataset(base_folder):
    """
    Carga imágenes y etiquetas desde la carpeta base 'Data'.
    """
    images = []
    labels = []

    for category in sorted(os.listdir(base_folder)):
        category_path = os.path.join(base_folder, category)
        if not os.path.isdir(category_path):
            continue

        for label in sorted(os.listdir(category_path)):
            label_path = os.path.join(category_path, label)
            if not os.path.isdir(label_path):
                continue

            for file in os.listdir(label_path):
                img_path = os.path.join(label_path, file)
                img = preprocess_image(img_path)
                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Cargar dataset
X, y = load_dataset("Data")
print("Total imágenes cargadas:", len(X))

# Visualización aleatoria
idx = random.randint(0, len(X)-1)
plt.imshow(X[idx], cmap='gray')
plt.title(f"Etiqueta: {y[idx]}")
plt.axis('off')
plt.show()


In [ ]:
def segment_characters(image):
    """
    Segmenta caracteres de la imagen y los centra en 28x28.
    """
    img = (image * 255).astype(np.uint8)
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    char_images = []
    bounding_boxes = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w < 5 or h < 5:
            continue

        roi = img[y:y+h, x:x+w]

        char_img = np.zeros((28,28), dtype=np.float32)

        # Mantener proporción y escalar a máximo 20 píxeles
        scale = 20.0 / max(w, h)
        new_w = int(w * scale)
        new_h = int(h * scale)
        roi_resized = cv2.resize(roi, (new_w, new_h))

        x_offset = (28 - new_w) // 2
        y_offset = (28 - new_h) // 2

        char_img[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = roi_resized / 255.0

        char_images.append(char_img)
        bounding_boxes.append((x, y, w, h))

    # Ordenar de izquierda a derecha
    char_images = [x for _, x in sorted(zip(bounding_boxes, char_images), key=lambda b: b[0][0])]
    return char_images


In [ ]:
# Carpeta donde colocarás las imágenes de prueba
test_folder = "TestImages"

# Buscar todas las imágenes en la carpeta
img_files = sorted(glob.glob(f"{test_folder}/*.*"))

if not img_files:
    raise FileNotFoundError(f"No se encontraron imágenes en {test_folder}")

# Mostrar lista de imágenes disponibles
print("Imágenes disponibles para reconocimiento:")
for i, f in enumerate(img_files):
    print(f"{i}: {f}")

# Selección por índice
idx = int(input(f"Selecciona el índice de la imagen a procesar (0-{len(img_files)-1}): "))
test_img_path = img_files[idx]
print("Procesando imagen:", test_img_path)

# Cargar y binarizar
img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)
_, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
img_bin = img_bin / 255.0

# Segmentar caracteres
chars = segment_characters(img_bin)

# Mostrar caracteres segmentados
plt.figure(figsize=(10,2))
for i, c in enumerate(chars):
    plt.subplot(1, len(chars), i+1)
    plt.imshow(c, cmap='gray')
    plt.axis('off')
plt.show()


In [ ]:
def recognize_character(char_img, dataset_X, dataset_y):
    """
    Reconoce un carácter usando correlación normalizada.
    """
    best_label = None
    best_score = -1

    char_vec = char_img.flatten()
    char_norm = np.linalg.norm(char_vec)
    if char_norm == 0:
        char_norm = 1e-6

    for i, ref_img in enumerate(dataset_X):
        ref_vec = ref_img.flatten()
        ref_norm = np.linalg.norm(ref_vec)
        if ref_norm == 0:
            ref_norm = 1e-6

        score = np.dot(char_vec, ref_vec) / (char_norm * ref_norm)

        if score > best_score:
            best_score = score
            best_label = dataset_y[i]

    return best_label

def recognize_text(char_images, dataset_X, dataset_y):
    """
    Reconoce todos los caracteres segmentados y devuelve el texto completo.
    """
    text = ""
    for c in char_images:
        text += recognize_character(c, dataset_X, dataset_y)
    return text

# Reconocer texto
texto_reconocido = recognize_text(chars, X, y)
print("Texto reconocido:", texto_reconocido)
